# Submission 2: Sistem Prediksi Penyakit Jantung
Nama: Abdul Latif

Username dicoding: latif_abdul

| | Deskripsi |
| ----------- | ----------- |
| Dataset | [Heart Desease Prediction](https://www.kaggle.com/datasets/mexwell/heart-disease-dataset) |
| Masalah | Penyakit jantung merupakan salah satu penyakit kronis yang menjadi penyebab kematian terbanyak di dunia. Menurut data <i>World Health Organization (WHO)</i> pada tahun 2019, penyakit jantung bertanggung jawab atas 17,9 juta kematian, atau sekitar 32% dari total kematian global. Di Indonesia, penyakit jantung juga menjadi salah satu penyakit terbanyak yang menyebabkan kematian. Berdasarkan data Riskesdas tahun 2018, prevalensi penyakit jantung di Indonesia mencapai 7,3%. <b>Himpunan data ini terdiri dari 1190 instans dengan 11 fitur. Kumpulan data ini dikumpulkan dan digabungkan di satu tempat untuk membantu memajukan penelitian tentang pembelajaran mesin terkait CAD dan algoritma penambangan data, dan mudah-mudahan pada akhirnya dapat memajukan diagnosis klinis dan perawatan dini. Dengan menggunakan dataset ini diharapkan bisa menjadi pendukung dibuatnya sistem untuk memprediksi seseorang memiliki penyakit jantung atau tidak</b> |
| Solusi machine learning | Berdasarkan masalah tersebut dibutuhkan sistem machine learning yang bisa memprediksi seseorang memiliki penyakit jantung atau tidak |
| Metode pengolahan | 1. Pengumpulan Data :<br> Sumber Data: kaggle <br>2. Pra-pemrosesan Data:<br>Ekstraksi Fitur: Fitur-fitur pada data diekstraksi.<br>Representasi Numerik: Fitur yang berupa teks diubah menjadi representasi numerik yang dapat dipahami oleh model machine learning.<br>3. Pembuatan Model Machine Learning:<br>Pilihan Algoritma: Algoritma machine learning yang umum digunakan untuk klasifikasi adalah <i>Deep learning</i><br>Pelatihan Model: Model machine learning dilatih menggunakan data yang telah dipraproses.<br>Penyesuaian Hiperparameter: Parameter model machine learning dioptimalkan untuk mencapai kinerja terbaik dalam mengklasifikasikan data baru.<br>4. Evaluasi Model:<br>Metrik Evaluasi: Metrik evaluasi seperti akurasi, presisi, dan recall digunakan untuk mengukur kinerja model machine learning pada data uji.<br>Penyesuaian Model: Jika model tidak menunjukkan kinerja yang memuaskan, model perlu disesuaikan dengan mengubah algoritma, hiperparameter, atau data pelatihan. <br>Pemantauan dan Pembaruan: Kinerja model machine learning perlu dipantau secara berkala dan diperbarui dengan data baru untuk menjaga akurasinya. |
| Arsitektur model | arsitektur model yang digunakan adalah input layer dan 4 layer dense dan setelah proses tuning didapatkan learning rate terbaik yaitu 0.01 dan epoch terbaik 2 |
| Metrik evaluasi | metrik yang digunakan yaitu binary accuracy |
| Performa model | performa model yang dihasilkan cukup baik dengan binary_akurasi 99% dan val_binary_accuracy 79% |
| Opsi Deployment | Model dideploy ke server cloud cloudeka |
| Web App | http://103.190.215.239:8501/v1/models/cc-model/metadata |
| Hasil Monitoring | Hasil monitoring menunjukkan nilai performa model 2961 |

In [1]:
import os
from typing import Text
from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

2024-04-27 23:25:34.006784: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
PIPELINE_NAME = "latif_abdul-pipeline"
 
# pipeline inputs
DATA_ROOT = "data"
TRANSFORM_MODULE_FILE = "modules/heart_desease_transform.py"
TRAINER_MODULE_FILE = "modules/heart_desease_trainer.py"
TUNER_MODULE_FILE = "modules/heart_desease_tuner.py"
# requirement_file = os.path.join(root, "requirements.txt")
 
# pipeline outputs
OUTPUT_BASE = "output"

In [3]:
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

In [4]:
def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:
    
    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing"
        # 0 auto-detect based on on the number of CPUs available 
        # during execution time.
        "----direct_num_workers=0" 
    ]
    
    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        eam_pipeline_args=beam_args
    )

In [5]:
logging.set_verbosity(logging.INFO)
    
from modules.components import init_components

components = init_components(
    DATA_ROOT,
    training_module=TRAINER_MODULE_FILE,
    transform_module=TRANSFORM_MODULE_FILE,
    tuner_module=TUNER_MODULE_FILE,
    training_steps=5000,
    eval_steps=1000,
    serving_model_dir=serving_model_dir,
)

pipeline = init_local_pipeline(components, pipeline_root)
BeamDagRunner().run(pipeline=pipeline)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Pipeline root set to: output/heart-desease-pipeline
INFO:absl:Generating ephemeral wheel package for '/Users/zesthub/Downloads/a443-cc-pipeline/modules/heart_desease_transform.py' (including modules: ['heart_desease_transform', 'heart_desease_tuner', 'heart_desease_trainer', 'components']).
INFO:absl:User module package has hash fingerprint version 608bffbaab472a0411f23f2edc6f34c7b9de15d02db91ae6dbd15056e2c3caf1.
INFO:absl:Executing: ['/opt/anaconda3/envs/mlops-tfx/bin/python', '/var/folders/4x/xyk3ydbd7y1f6bxvqv9rnwg80000gn/T/tmp1egxmlpr/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/var/folders/4x/xyk3ydbd7y1f6bxvqv9rnwg80000gn/T/tmpgd6xji26', '--dist-dir', '/var/folders/4x/xyk3ydbd7y1f6bxvqv9rnwg80000gn/T/tmpql9abxoy']


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying heart_desease_transform.py -> build/lib
copying heart_desease_tuner.py -> build/lib
copying heart_desease_trainer.py -> build/lib
copying components.py -> build/lib
installing to /var/folders/4x/xyk3ydbd7y1f6bxvqv9rnwg80000gn/T/tmpgd6xji26
running install
running install_lib
copying build/lib/heart_desease_transform.py -> /var/folders/4x/xyk3ydbd7y1f6bxvqv9rnwg80000gn/T/tmpgd6xji26
copying build/lib/heart_desease_tuner.py -> /var/folders/4x/xyk3ydbd7y1f6bxvqv9rnwg80000gn/T/tmpgd6xji26
copying build/lib/heart_desease_trainer.py -> /var/folders/4x/xyk3ydbd7y1f6bxvqv9rnwg80000gn/T/tmpgd6xji26
copying build/lib/components.py -> /var/folders/4x/xyk3ydbd7y1f6bxvqv9rnwg80000gn/T/tmpgd6xji26
running install_egg_info


/opt/anaconda3/envs/mlops-tfx/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
INFO:absl:Successfully built user code wheel distribution at 'output/heart-desease-pipeline/_wheels/tfx_user_code_Transform-0.0+608bffbaab472a0411f23f2edc6f34c7b9de15d02db91ae6dbd15056e2c3caf1-py3-none-any.whl'; target user module is 'heart_desease_transform'.
INFO:absl:Full user module path is 'heart_desease_transform@output/heart-desease-pipeline/_wheels/tfx_user_code_Transform-0.0+608bffbaab472a

running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /var/folders/4x/xyk3ydbd7y1f6bxvqv9rnwg80000gn/T/tmpgd6xji26/tfx_user_code_Transform-0.0+608bffbaab472a0411f23f2edc6f34c7b9de15d02db91ae6dbd15056e2c3caf1-py3.9.egg-info
running install_scripts
creating /var/folders/4x/xyk3ydbd7y1f6bxvqv9rnwg80000gn/T/tmpgd6xji26/tfx_user_code_Transform-0.0+608bffbaab472a0411f23f2edc6f34c7b9de15d02db91ae6dbd15056e2c3caf1.dist-info/WHEEL
creating '/var/folders/4x/xyk3ydbd7y1f6bxvqv9rnwg80000gn/T/tmpql9abxoy/tfx_user_code_Transform-0.0+608bffbaab472

/opt/anaconda3/envs/mlops-tfx/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
INFO:absl:Successfully built user code wheel distribution at 'output/heart-desease-pipeline/_wheels/tfx_user_code_Trainer-0.0+608bffbaab472a0411f23f2edc6f34c7b9de15d02db91ae6dbd15056e2c3caf1-py3-none-any.whl'; target user module is 'heart_desease_trainer'.
INFO:absl:Full user module path is 'heart_desease_trainer@output/heart-desease-pipeline/_wheels/tfx_user_code_Trainer-0.0+608bffbaab472a0411f23f

...

INFO:absl:Using output/heart-desease-pipeline/Trainer/model/38/Format-Serving as baseline model.


INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'fairness_indicator_thresholds': 'null', 'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "AUC"\n        },\n        {\n          "class_name": "Precision"\n        },\n        {\n          "class_name": "Recall"\n        },\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "BinaryAccuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 0.0001,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.5\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "target"\n    }\n  ],\n  "slicing_specs": [\n    {}\n  ]\n}', 'example_splits': 'null'} 'custom_extractors'


INFO:absl:Evaluation complete. Results written to output/heart-desease-pipeline/Evaluator/evaluation/50.
INFO:absl:Checking validation results.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Blessing result False written to output/heart-desease-pipeline/Evaluator/blessing/50.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 50 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'blessing': [Artifact(artifact: uri: "output/heart-desease-pipeline/Evaluator/blessing/50"
, artifact_type: name: "ModelBlessing"
)], 'evaluation': [Artifact(artifact: uri: "output/heart-desease-pipeline/Evaluator/evaluation/50"
, artifact_type: name: "ModelEvaluation"
)]}) for execution 50
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:node Evaluator is finished.
INFO:absl:node Pusher is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.pusher.component.Pusher"
    base_type: DEPLOY
  }
  id: "Pusher"
}
contexts {
  contexts {
    type {
      name: "pipeline"
